In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import KFold

# Carregando base de dados  pré-processada

In [2]:
dataset = pd.read_csv("../../Database/dataBaseWithNER.csv")
dataset = dataset.drop(columns=["Unnamed: 0"])
dataset = dataset.dropna()
target = np.array(dataset["target"].array)

dataset

,email,target
0,start date hourahead timee cardinall hou...,0
1,service long desk price structure deal quote ...,0
2,start date cardinall hourahead timee card...,0
3,start date hourahead timee cardinall anc...,0
4,cardinall deliverable revenue management marke...,0
...,...,...
33340,bio matrix scientific group symbo bmxg p...,1
33341,cardinall step away hot naked webcam girl liv...,1
33342,need pill increase performance click seroius ...,1
33343,datee final nom inlet hpl eastrans car...,0


In [3]:
emailsText = []
for email in dataset["email"]:
    emailsText.append(email)

del dataset

In [4]:
print(len(emailsText))

33341


# Representação vetorial Bag of Words

In [5]:
vectorizer = CountVectorizer(max_features=2100)
X = vectorizer.fit_transform(emailsText)

bag = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

del emailsText
del X

bag

E:\DevPack\anaconda3\envs\data_science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,ability,able,absolutely,abuse,accept,acceptance,accepted,access,according,...,xanax,xl,xp,yahoo,year,yes,yield,yo,young,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33336,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
33337,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
33338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
bag = np.array(bag)
bag

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# Visualização de dados com TSNE

In [7]:
# model = TSNE(n_components=2, random_state=0)
# #model = PCA(n_components=50, svd_solver='full')
# array_red = model.fit_transform(bag)
#
# df_tsne = pd.DataFrame(array_red)
#
# df_tsne['Target'] = target
# df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]
#
# df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]
#
# plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')
#
# plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')
#
# plt.title('Dados')
# plt.xlabel('x')
# plt.ylabel('y')
#
# plt.show()

In [8]:
bag = bag.reshape((bag.shape[0], 1, bag.shape[1]))

print(bag.shape)

(33341, 1, 2100)


In [9]:
foldsAccuracy = []
foldLosses = []

In [10]:
kfold = KFold(n_splits=4, shuffle=True)

In [11]:
foldCount = 1
for train, test in kfold.split(bag, target):
    model = keras.models.Sequential([
        keras.layers.LSTM(50, activation='tanh',recurrent_activation='sigmoid',input_shape=[1, bag.shape[2]]),

        keras.layers.Dense(len(set(target)), activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

    print('****************************************************')
    print(f'Iniciando treinamento da fold: {foldCount}.')

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

    history = model.fit(bag[train], target[train], epochs=200, callbacks=callbacks, validation_split=0.05)

    scores = model.evaluate(bag[test], target[test], verbose=0)
    print(f'Score fold {foldCount}: {model.metrics_names[0]} de {scores[0]}; {model.metrics_names[1]} de {scores[1]*100}%')

    foldsAccuracy.append(scores[1] * 100)
    foldLosses.append(scores[0])

    foldCount = foldCount + 1

****************************************************
Iniciando treinamento da fold: 1.
Epoch 1/200
743/743 [==============================] - 12s 12ms/step - loss: 0.6928 - accuracy: 0.5094 - val_loss: 0.6929 - val_accuracy: 0.5076 - lr: 0.0100
Epoch 2/200
743/743 [==============================] - 10s 13ms/step - loss: 0.6920 - accuracy: 0.5148 - val_loss: 0.6937 - val_accuracy: 0.4868 - lr: 0.0100
Epoch 3/200
743/743 [==============================] - 8s 11ms/step - loss: 0.6913 - accuracy: 0.5112 - val_loss: 0.6926 - val_accuracy: 0.4884 - lr: 0.0100
Epoch 4/200
743/743 [==============================] - 8s 11ms/step - loss: 0.6908 - accuracy: 0.5147 - val_loss: 0.6914 - val_accuracy: 0.5827 - lr: 0.0100
Epoch 5/200
743/743 [==============================] - 8s 11ms/step - loss: 0.6904 - accuracy: 0.5280 - val_loss: 0.6916 - val_accuracy: 0.4916 - lr: 0.0100
Epoch 6/200
743/743 [==============================] - 9s 12ms/step - loss: 0.6900 - accuracy: 0.5198 - val_loss: 0.6910 - val

In [12]:
print('****************************************************')
print('Score de cada fold:')
for i in range(0, len(foldsAccuracy)):
    print('****************************************************')
    print(f'--> Fold {i+1}: Loss: {foldLosses[i]} ; Accuracy: {foldsAccuracy[i]}%')

print('****************************************************')
print('Média de accuracy das folds:')
print(f'--> Accuracy: {np.mean(foldsAccuracy)} (+- {np.std(foldsAccuracy)})')
print(f'--> Loss: {np.mean(foldLosses)}')
print('****************************************************')

****************************************************
Score de cada fold:
****************************************************
--> Fold 1: Loss: 0.15225523710250854 ; Accuracy: 94.93762254714966%
****************************************************
--> Fold 2: Loss: 0.10669742524623871 ; Accuracy: 96.2207555770874%
****************************************************
--> Fold 3: Loss: 0.07980623096227646 ; Accuracy: 97.6604700088501%
****************************************************
--> Fold 4: Loss: 0.09429062157869339 ; Accuracy: 97.15656638145447%
****************************************************
Média de accuracy das folds:
--> Accuracy: 96.4938536286354 (+- 1.036414455716904)
--> Loss: 0.10826237872242928
****************************************************


# Validação

In [13]:
#categories = ["Ham", "Spam"]
#
#skplt.metrics.plot_confusion_matrix(
#    [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#    title="Confusion Matrix",
#    cmap="Purples",
#    hide_zeros=True,
#    figsize=(5,5)
#)
#
#plt.xticks()

In [14]:
# skplt.metrics.plot_confusion_matrix(
#     [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#     normalize=True,
#     title="Confusion Matrix",
#     cmap="Purples",
#     hide_zeros=True,
#     figsize=(5,5)
# )
#
# plt.xticks()